# Assignment: Unix

### Wouter Zorgdrager and Georgios Gousios

As we have seen during the lecture, the Unix command line is extremely
versatile. This comes at a cost: you need to know which tool is suitable for the
job or to build it. For this assignment, you will have to use a Unix command line 
(i.e. in the course's VM, a Mac or Ubuntu for Windows) 
to accomplish the indicated tasks. Not all tools you
will need are readily available in the default installation of Xubuntu that we 
are using in the course. You need to search online (or implement!) for 
the appropriate tools and command line options.

_Note_: To develop and run those commands in Jupyter, you need to install the
[Bash kernel](https://github.com/takluyver/bash_kernel). Alternatively, you
can create a file where you paste all programs/pipelines, print it and upload
it to the assignment directory.

There **120** points to collect. To get a 10, you need **110**.

## Shell programming

**Q1 (10 points)** Write a pipeline that converts (recursively!) a directory
structure full of `.wav` files to `.mp3`s.

In [ ]:
find . -type f -name "*.wav" | xargs -P 2 -I {} lame {}

**Q2 (10 points)** Write a program that given a directory structure of text files, it
prints the 10 most common words in those files (across all files).

The output should look like this (example is for 3 most common items):

``
5 most_common_word
2 less_common_word
1 least_common_word
``

In [ ]:
find . -type f -name "*.txt" | xargs cat | tr ' ' '\n' | sort | uniq -c | sort -nr | head -10

**Q3 (10 points)** Given this
[dataset](https://drive.google.com/file/d/1HYkiUFb_NKw4BWXSOvjgtw0jnjF4eA2s/view?usp=sharing)
(which you will also be using later on), write a program that obtains the star
count, watcher count and number of forks for first 10 repositories from the GitHub API.
The output must look like the following:

```
url,stars,watchers,forks
https://api.github.com/repos/8d8d/Think.Admin,0,0,0
https://api.github.com/repos/971638267/RetrofitAndRxjavaforRecyclerview,12,12,3
https://api.github.com/repos/9alsacelost/greenDAO,0,0,0
https://api.github.com/repos/a1265137718/ZoomHeader,0,0,0
https://api.github.com/repos/12345678/NonExistingRepo,,,
```

**Note** Make sure you use this path for the csv file: `repositories.csv` otherwise auto-grading will fail.

**Warning** The GitHub API requires authentication, otherwise it is limited to 60 requests per hour. Make sure you setup [an OAuth key](https://developer.github.com/v3/#oauth2-token-sent-in-a-header) and use this key for doing the requests. After the assignment deadline, you can remove it. _If you fail to submit a solution including a key, the grading will fail_

In [1]:
for i in `head -n 11 repositories.csv | cut -d ',' -f2 | tail -n +2`; do printf "$i," ; curl -s $i?access_token=90c45cdbeabd2601e16bfe9d535faf3f5bd38db9 | jq '.stargazers_count, .watchers_count, .forks_count' | tr -d '"' | paste -sd, -| sed 's/null//g'
done

head: cannot open 'repositories.csv' for reading: No such file or directory


**Q4 (10 points)** Write a program that downloads all JPEG pictures in a web
page. JPEG pictures are identified by the extensions `*.jpg` and `*.jpeg`.
Your program must accept the URL to process as an argument. For example:

```bash
$ jpegdl https://www.nu.nl
[...]
$ ls
zi7xo0pa6h6x_std320.jpg
hsnxkcyat176_std320.jpg
[...]
```

In [ ]:
curl -s $1 | grep img | cut -f2 -d '"' | egrep "jpg|jpeg" | xargs wget

**Q5 (10 points)** All Unix systems have a
[dictionary file](https://en.wikipedia.org/wiki/Words_(Unix)) residing under
`/usr/share/dict/words` or `/usr/dict/words`. Use it to implement a (rudimentary)
spellchecker. Your spellchecker should read a file named `foo.txt` and print
a list all the words in the document to be checked that are not in the dictionary.
An example usage session can be seen below.

```bash
$ cat foo.txt
I am a nicelly formatted sentence,
but I contain errors.

$ cat foo.txt | spellchecker
nicelly
formated
```

In [1]:
cat foo.txt | 
tr [:upper:] [:lower:] |
tr ' ' '\n' |
tr -d ',' | 
tr -d '.' | 
sort | 
uniq |
comm -23 - /usr/share/dict/words

cat: foo.txt: No such file or directory
comm: file 2 is not in sorted order


: 1

**Q6 (10 points)** Given [this repository](https://github.com/SERG-Delft/jpacman-framework)
at commit [05681455d905586f940e0e00e](https://github.com/SERG-Delft/jpacman-framework/commit/05681455d905586f940e0e00ed05f34266332ae9), find the sizes for all versions of all test files (assume that all test files are under `src/test/java`). The output must look like the following:

```
blob_id blob_path size_in_bytes
```

for example:

```
da14c3975e src/test/java/nl/tudelft/jpacman/npc/ghost/NavigationTest.java 165
1fbe0d836d src/test/java/nl/tudelft/jpacman/sprite/SpriteTest.java 108
3bde982975 src/test/java/nl/tudelft/jpacman/sprite/SpriteTest.java 108
6286792e43 src/test/java/nl/tudelft/jpacman/sprite/SpriteTest.java 110
```

A blob in Git represents a file version.

/home/bigdata/Downloads/assignment_1


## Data processing

For the assignments in this section, we use the following [dataset](https://drive.google.com/file/d/1HYkiUFb_NKw4BWXSOvjgtw0jnjF4eA2s/view?usp=sharing) containing repositories.

Write pipelines to calculate answers to the following questions:

_Note:_ Use `"repositories.csv"` as filename, otherwise automatic grading will fail.

**Q7 (10 points)** Count the number of repositories written in `Java`.

In [ ]:
cd /home/bigdata/datasets/

awk -F ',' '{print$5}' repositories.csv | sort | uniq -c | grep -w Java | tr -dc '0-9'

**Q8 (10 points)** How many repositories were forked _and_ written in PHP?

In [ ]:
cd /home/bigdata/datasets/

awk -F "\"*,\"*" '{print $5","$7}' repositories.csv | grep -w PHP | grep -v NULL | wc -l

**Q9 (10 points)** Which `owner_id` owns most repositories?

In [ ]:
cd /home/bigdata/datasets/

awk -F ',' '{print$3}' repositories.csv | sort | uniq -c | sort -nr | head -1 | cut -d ' ' -f 8

**Q10 (10 points)**  Which repositories are created between `01-01-2017` and `24-03-2017` (both inclusive)? Print the names.

In [ ]:
cd /home/bigdata/datasets/

awk -F "\"*,\"*" '{print $4" "$6}' repositories.csv | cut -d ' ' -f 1,2 | sort -k2 -n | egrep '2017-(0[1-2]-([0-3][0-9])|03-(0[1-9]|1[0-9]|2[0-4]))'  | cut -d ' ' -f 1

**Q11 (10 points)**: Print the `10` most used programming languages sorted on popularity.

In [ ]:
cd /home/bigdata/datasets/

awk -F ',' '{print$5}' repositories.csv | sort | uniq -c | sort -nr | grep -v NULL | head -10 | tr -d '0-9' | tr -d ' '

**Q12 (10 points)**: Print the `username` of the owners whose repositories are deleted.

_Hint_: Use the url field.

In [2]:
cd /home/bigdata/datasets/

awk -F "\"*,\"*" '{print $2" "$8}' repositories.csv | grep -vw 0 | cut -d ' ' -f 1 | cut -d '/' -f 5 | grep -v url

airani
AlloyTeam
amol-mandhane
bircher
bit101
chergert
devinbarry
DiUS
dmwm
drewgriffith15
kleer-la
remijean
wujunchuan
